In [1]:
import numpy as np
import numpy.random as npr
from scipy import sparse
from scipy.spatial.distance import cosine
from time import time

**Storage in the network model**

In [75]:
def get_one(n):
    """Return array of ones of length n."""
    return np.ones(n)

def get_weights_rand(shape, prob):
    """Return a weights matrix with prob amount of random connections activated.
    
    Args:
        shape: (int) number of neurons in network
        prob: (float) fraction of connections to be activated
    
    Returns:
        weights: (numpy matrix) connectivity weight matrix
    """
    # THIS IS NOT QUITE RANDOM...
    l = int(shape*shape*prob)
    rows = npr.choice(range(shape), l)
    cols = npr.choice(range(shape), l)
    weights = sparse.csr_matrix((np.ones(l), (rows, cols)), shape=(shape,shape))
    weights.setdiag(0)
    return weights

def get_patterns(activity, num_neurons, num_patterns):
    """Return a matrix where each row is a pattern with activity level activity.
    
    Args:
        activity: (float) avg activity level per pattern
        num_neurons: (int) num neurons in network
        num_patterns: (int) num patterns to generate
    
    Returns:
        patterns: (numpy matrix) set of patterns shape=(num_patterns, num_neurons)
    """
#     l = int(num_neurons*activity)
#     rows = npr.choice(range(num_patterns), l)
#     cols = npr.choice(range(num_neurons), l)
#     patterns = sparse.csr_matrix((np.ones(l), (rows, cols)), shape=(num_patterns, num_neurons))
    return npr.binomial(1, activity, size=(num_patterns, num_neurons))

def get_syn_weights(patterns):
    """Return synaptic weight matrix.
    
    Args:
        patterns: (numpy matrix) set of patterns, shape=(num_patterns, num_neurons)
        
    Returns:
        syn: (numpy matrix) synaptic weight matrix, shape=(num_neurons, num_neurons)
    """
    num_patterns, num_neurons = patterns.shape
    row_ind = []
    col_ind = []
    for i in range(num_patterns):
        p = sparse.csr_matrix(patterns[i, :])
        mat = p.multiply(p.T)
        rows, cols = np.nonzero(mat)
        row_ind = np.concatenate([row_ind, rows])
        col_ind = np.concatenate([col_ind, cols])
    o = np.ones(len(row_ind))
    return sparse.csr_matrix((o, (row_ind, col_ind)), shape=(num_neurons, num_neurons))

# everything is decreased by 1 order of magnitude from paper

n = 1000                  # number of neurons in network; CA3 is 330,000 (or double)
p = .5                   # connection probability for weight matrix
m = 5                   # number of patterns to store
f = 0.005                  # average activity level of each pattern

now = time()
W = get_weights_rand(n, p) # connectivity weight matrix; are neurons connected?
print('made W in', time() - now)
now = time()
M = get_patterns(f, n, m)  # patterns
print('made M in', time() - now)
now = time()
J = get_syn_weights(M)     # synaptic weight matrix J; how much are neurons connected?
print('made J in', time() - now)
now = time()

W.shape, M.shape, J.shape

sum(sum(W))

made W in 0.0874471664428711
made M in 0.00040793418884277344
made J in 0.0035719871520996094


/anaconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py:746: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


<1x1000 sparse matrix of type '<class 'numpy.float64'>'
	with 1000 stored elements in Compressed Sparse Row format>

In [56]:
# TOO SLOW!!
print('W symmetric?', W!=W.T)
print('should be no')
print('J symmetric?', J!=J.T)
print('should be yes')

W symmetric?   (0, 38)	True
  (0, 50)	True
  (0, 54)	True
  (0, 55)	True
  (0, 74)	True
  (0, 98)	True
  (0, 170)	True
  (0, 191)	True
  (0, 221)	True
  (0, 238)	True
  (0, 243)	True
  (0, 256)	True
  (0, 258)	True
  (0, 263)	True
  (0, 269)	True
  (0, 270)	True
  (0, 282)	True
  (0, 292)	True
  (0, 295)	True
  (0, 325)	True
  (0, 330)	True
  (0, 335)	True
  (0, 337)	True
  (0, 346)	True
  (0, 374)	True
  :	:
  (9999, 9746)	True
  (9999, 9750)	True
  (9999, 9768)	True
  (9999, 9785)	True
  (9999, 9794)	True
  (9999, 9795)	True
  (9999, 9822)	True
  (9999, 9828)	True
  (9999, 9831)	True
  (9999, 9837)	True
  (9999, 9851)	True
  (9999, 9874)	True
  (9999, 9880)	True
  (9999, 9884)	True
  (9999, 9889)	True
  (9999, 9891)	True
  (9999, 9902)	True
  (9999, 9918)	True
  (9999, 9931)	True
  (9999, 9933)	True
  (9999, 9938)	True
  (9999, 9959)	True
  (9999, 9986)	True
  (9999, 9988)	True
  (9999, 9994)	True
should be no
J symmetric? 
should be yes


**Recall in the network model**

In [82]:
def get_degraded_pattern(pattern, valid, spurious):
    """Return degraded pattern for input to simulation to do pattern completion.
    
    Args:
        pattern: (array) pattern to degrade, 0s or 1s, shape=(num_neurons)
        valid: (float) fraction of valid firing neurons
        spurious: (float) fraction of spurious firing neurons
    
    Return:
        deg_pat: (array) degraded pattern, 0s or 1s, shape=(num_neurons)
    """
#     print('pattern of len {} with {} fires'.format(len(pattern), sum(pattern)))
    fires = np.nonzero(pattern)[0]
    deg_fires = npr.choice(fires, size=int(valid*len(fires)), replace=False)

    nonfires = np.where(pattern == 0)[0]
    spur_fires = npr.choice(nonfires, size=int(spurious*len(nonfires)), replace=False)
    
    all_fires = np.concatenate((deg_fires, spur_fires), axis=0)

    deg_pat = np.zeros(shape=(len(pattern),))
    deg_pat[all_fires] = 1
    
    return deg_pat


def simulate(in_pattern, out_pattern, con_mat, syn_mat, g0, g1, cycles=10, pprint=False):
    """Return final matrix of updates.
    
    Args:
        in_pattern: (array) degraded pattern to start
        out_pattern: (array) original pattern to attempt to recall
        con_mat: (matrix) connectivity matrix
        syn_mat: (matrix) synaptic weights matrix
        g0: (float) firing threshold
        g1: (float) inhibition factor
        cycles: (int) num times to run iteration
    
    Return:
        curr_mat: (matrix) end state matrix
    """
    n = len(in_pattern)
    W_J = con_mat.multiply(syn_mat)
#     print('W_J', len(W_J.nonzero()))
    state = sparse.csr_matrix(np.matrix(in_pattern)).T
    if pprint:
        print('{:<10} {:<10} {:<10}'.format('steps', 'correl', 'num fires'))
    for i in range(cycles):
        if pprint:
            curr = state.T.toarray()[0]
            print('{:<10} {:<10.2f} {:<10}'.format(i, get_correlation(curr, out_pattern), sum(curr)))
        _inter = W_J.dot(state)
        h = (1/n) * _inter
#         print('h', h)
        condition = h - ( (1/n) * g1 * np.sum(state) )
        state = condition > g0
    curr = state.T.toarray()[0]
    return get_correlation(curr, out_pattern)

def get_correlation(p1, p2):
    p1_ = p1 - (sum(p1)/len(p1))
    p2_ = p2 - (sum(p2)/len(p2))
    return 1 - cosine(p1_, p2_)

b_vald = .5  # fraction of valid firing neurons in patterns
b_spur = 0.001  # fraction of spurious firing neurons in patterns


now = time()
godpat = M[0, :]
badpat = get_degraded_pattern(godpat, b_vald, b_spur)
print('made degraded pattern in', time() - now)
print('correlation', get_correlation(godpat, badpat))

g_0 = 7 * 10**-6
g_1 = 0.0
res = simulate(badpat, godpat, W, J, g_0, g_1, pprint=True)
# should see a correlation between original and degraded patterns of 0.71

made degraded pattern in 0.0005190372467041016
correlation 0.604132328572
steps      correl     num fires 
0          0.60       15.0      
1          0.77       37        
2          0.67       49        
3          0.67       49        
4          0.67       49        
5          0.67       49        
6          0.67       49        
7          0.67       49        
8          0.67       49        
9          0.67       49        


In [83]:
n = 5000                  # number of neurons in network; CA3 is 330,000 (or double)
p = .7                   # connection probability for weight matrix
m = 5                   # number of patterns to store
f = 0.005                  # average activity level of each pattern

b_vald = .5  # fraction of valid firing neurons in patterns
b_spur = 0.00  # fraction of spurious firing neurons in patterns

g_0 = 7 * 10**-6
g_1 = 0.0

p_range = [.1, .3, .5, .7]
m_range = [5, 10]

res_mat = np.zeros(shape=(len(p_range),len(m_range)))

for i, p in enumerate(p_range):
    for j, m in enumerate(m_range):
        print(i,j, end="...")
        res = 0
        for k in range(5):
            W = get_weights_rand(n, p) # connectivity weight matrix; are neurons connected?
            M = get_patterns(f, n, m)  # patterns
            J = get_syn_weights(M)     # synaptic weight matrix J; how much are neurons connected?

            godpat = M[0, :]
            badpat = get_degraded_pattern(godpat, b_vald, b_spur)

            res += simulate(badpat, godpat, W, J, g_0, g_1)
        res_mat[i, j] = res/5
print()
print(p_range)
print(m_range)
res_mat

0 0...

/anaconda3/lib/python3.6/site-packages/scipy/sparse/compressed.py:746: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  SparseEfficiencyWarning)


0 1...1 0...1 1...2 0...2 1...3 0...3 1...
[0.1, 0.3, 0.5, 0.7]
[5, 10]


array([[ 0.92363989,  0.81686913,  0.        ,  0.        ,  0.        ],
       [ 0.7971595 ,  0.52337874,  0.        ,  0.        ,  0.        ],
       [ 0.72830927,  0.71143631,  0.        ,  0.        ,  0.        ],
       [ 0.8472593 ,  0.56493426,  0.        ,  0.        ,  0.        ],
       [ 0.        ,  0.        ,  0.        ,  0.        ,  0.        ]])

In [37]:
x = np.matrix([1,1,0,0])
print(x)
y = sparse.csr_matrix(x)
print(y.get_shape())
print(y.T.get_shape())
z = y.multiply(y.T)
print(z.get_shape())
z.toarray()

[[1 1 0 0]]
(1, 4)
(4, 1)
(4, 4)


array([[1, 1, 0, 0],
       [1, 1, 0, 0],
       [0, 0, 0, 0],
       [0, 0, 0, 0]], dtype=int64)